In [1]:
import pyspark

In [2]:
pyspark.__file__

'/usr/local/lib/python3.9/site-packages/pyspark/__init__.py'

In [3]:
from pyspark.sql import SparkSession

In [4]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/03/03 21:11:12 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [5]:
spark.version

'3.5.4'

In [10]:
!curl -o yellow_tripdata_2024-10.parquet https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-10.parquet

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 61.3M  100 61.3M    0     0  40.7M      0  0:00:01  0:00:01 --:--:-- 40.8M


In [11]:
df = spark.read \
    .parquet('yellow_tripdata_2024-10.parquet')

In [26]:
df = df.repartition(4)

In [16]:
df.write.parquet('yellow_tripdata_2024-10_partition')

In [18]:
import pandas as pd

In [20]:
df.schema

StructType([StructField('VendorID', IntegerType(), True), StructField('tpep_pickup_datetime', TimestampNTZType(), True), StructField('tpep_dropoff_datetime', TimestampNTZType(), True), StructField('passenger_count', LongType(), True), StructField('trip_distance', DoubleType(), True), StructField('RatecodeID', LongType(), True), StructField('store_and_fwd_flag', StringType(), True), StructField('PULocationID', IntegerType(), True), StructField('DOLocationID', IntegerType(), True), StructField('payment_type', LongType(), True), StructField('fare_amount', DoubleType(), True), StructField('extra', DoubleType(), True), StructField('mta_tax', DoubleType(), True), StructField('tip_amount', DoubleType(), True), StructField('tolls_amount', DoubleType(), True), StructField('improvement_surcharge', DoubleType(), True), StructField('total_amount', DoubleType(), True), StructField('congestion_surcharge', DoubleType(), True), StructField('Airport_fee', DoubleType(), True)])

In [22]:
from pyspark.sql import types

In [95]:
yellow_schema = types.StructType([
    types.StructField('VendorID', types.IntegerType(), True), 
    types.StructField('tpep_pickup_datetime', types.TimestampType(), True), 
    types.StructField('tpep_dropoff_datetime', types.TimestampType(), True), 
    types.StructField('passenger_count', types.IntegerType(), True), 
    types.StructField('trip_distance', types.DoubleType(), True), 
    types.StructField('RatecodeID', types.IntegerType(), True), 
    types.StructField('store_and_fwd_flag', types.StringType(), True), 
    types.StructField('PULocationID', types.IntegerType(), True), 
    types.StructField('DOLocationID', types.IntegerType(), True), 
    types.StructField('payment_type', types.IntegerType(), True), 
    types.StructField('fare_amount', types.DoubleType(), True), 
    types.StructField('extra', types.DoubleType(), True), 
    types.StructField('mta_tax', types.DoubleType(), True), 
    types.StructField('tip_amount', types.DoubleType(), True), 
    types.StructField('tolls_amount', types.DoubleType(), True), 
    types.StructField('improvement_surcharge', types.DoubleType(), True), 
    types.StructField('total_amount', types.DoubleType(), True), 
    types.StructField('congestion_surcharge', types.DoubleType(), True), 
    types.StructField('Airport_fee', types.DoubleType(), True)])

In [96]:
df = spark.read \
    .schema(yellow_schema) \
    .parquet('yellow_tripdata_2024-10.parquet')

In [97]:
df.registerTempTable('yellow_trips_data')

In [98]:
spark.sql('''
select 
    count(VendorID)
from
    yellow_trips_data
where
    date_trunc('day', tpep_pickup_datetime) = '2024-10-15'
''').show()

[Stage 99:====================================>                     (5 + 3) / 8]

+---------------+
|count(VendorID)|
+---------------+
|         128632|
+---------------+



In [99]:
spark.sql('''
select 
    timestampdiff(hour, tpep_pickup_datetime, tpep_dropoff_datetime) as duration
from
    yellow_trips_data
order by
    timestampdiff(hour, tpep_pickup_datetime, tpep_dropoff_datetime) desc
limit 1
''').show()

[Stage 102:===================================>                     (5 + 3) / 8]

+--------+
|duration|
+--------+
|     162|
+--------+



In [100]:
!curl taxi_zone_lookup.csv https://d37ci6vzurychx.cloudfront.net/misc/taxi_zone_lookup.csv

"LocationID","Borough","Zone","service_zone"
1,"EWR","Newark Airport","EWR"
2,"Queens","Jamaica Bay","Boro Zone"
3,"Bronx","Allerton/Pelham Gardens","Boro Zone"
4,"Manhattan","Alphabet City","Yellow Zone"
5,"Staten Island","Arden Heights","Boro Zone"
6,"Staten Island","Arrochar/Fort Wadsworth","Boro Zone"
7,"Queens","Astoria","Boro Zone"
8,"Queens","Astoria Park","Boro Zone"
9,"Queens","Auburndale","Boro Zone"
10,"Queens","Baisley Park","Boro Zone"
11,"Brooklyn","Bath Beach","Boro Zone"
12,"Manhattan","Battery Park","Yellow Zone"
13,"Manhattan","Battery Park City","Yellow Zone"
14,"Brooklyn","Bay Ridge","Boro Zone"
15,"Queens","Bay Terrace/Fort Totten","Boro Zone"
16,"Queens","Bayside","Boro Zone"
17,"Brooklyn","Bedford","Boro Zone"
18,"Bronx","Bedford Park","Boro Zone"
19,"Queens","Bellerose","Boro Zone"
20,"Bronx","Belmont","Boro Zone"
21,"Brooklyn","Bensonhurst East","Boro Zone"
22,"Brooklyn","Bensonhurst West","Boro Zone"
23,"Staten Island","Bloomfield/Emerson Hill","Boro Zone"
24,

In [101]:
zone_shema = types.StructType([
    types.StructField('LocationID', types.IntegerType(), True), 
    types.StructField('Borough', types.StringType(), True), 
    types.StructField('Zone', types.StringType(), True), 
    types.StructField('service_zone', types.StringType(), True)])

In [102]:
zone = spark.read \
    .option("header", "true") \
    .schema(zone_shema) \
    .csv('taxi_zone_lookup.csv')

In [118]:
df_result = df.join(zone, df.PULocationID == zone.LocationID, how='inner')

In [119]:
df_result.registerTempTable('result')

In [124]:
spark.sql('''
select Zone, count(VendorID) as num_rec
from result
group by 1
order by count(VendorID)
''').show()

[Stage 120:============================>                            (4 + 4) / 8]

+--------------------+-------+
|                Zone|num_rec|
+--------------------+-------+
|Governor's Island...|      1|
|       Rikers Island|      2|
|       Arden Heights|      2|
|         Jamaica Bay|      3|
| Green-Wood Cemetery|      3|
|Charleston/Totten...|      4|
|   Rossville/Woodrow|      4|
|       West Brighton|      4|
|Eltingville/Annad...|      4|
|       Port Richmond|      4|
|         Great Kills|      6|
|        Crotona Park|      6|
|Heartland Village...|      7|
|     Mariners Harbor|      7|
|Saint George/New ...|      9|
|             Oakwood|      9|
|       Broad Channel|     10|
|New Dorp/Midland ...|     10|
|         Westerleigh|     12|
|     Pelham Bay Park|     12|
+--------------------+-------+
only showing top 20 rows

